In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [3]:
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    text = "".join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text


def removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise


In [4]:
df = normalize_text(df)
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhhwaitin call
2,sadness,funeral ceremonygloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [5]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [6]:
X = df['sentiment'].isin(['happiness','sadness'])
df = df[X]

In [7]:
df['sentiment'] = df['sentiment'].replace({'sadness':0,'happiness':1})
df.head()

C:\Users\karan\AppData\Local\Temp\ipykernel_13624\1083041448.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0,'happiness':1})


,sentiment,content
1,0,layin n bed headache ughhhhwaitin call
2,0,funeral ceremonygloomy friday
6,0,sleep im not thinking old friend want married ...
8,0,charviray charlene love miss
9,0,kelcouch sorry least friday


In [8]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
import dagshub
dagshub.init(repo_owner='KARAN-sys141',repo_name='mini_mlops',mlflow=True)

mlflow.set_tracking_uri('https://dagshub.com/KARAN-sys141/mini-mlops.mlflow')

mlflow.set_experiment('Logistic Regression Baseline')

Initialized MLflow to track repo "KARAN-sys141/mini_mlops"

Repository KARAN-sys141/mini_mlops initialized!

<Experiment: artifact_location='mlflow-artifacts:/2c736ccef32c45f3b2d1f0356a4d591e', creation_time=1753594666173, experiment_id='0', last_update_time=1753594666173, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
# with mlflow.start_run():
    
#     mlflow.log_param('vectorizer', 'Bag of Words')
#     mlflow.log_param('num_features', 1000)
#     mlflow.log_param('test_size', 0.2)

#     model = LogisticRegression()
#     model.fit(X_train, y_train.values.ravel())  # ✅ FIXED

#     mlflow.log_param('model', 'Logistic Regression')

#     y_pred = model.predict(X_test)

#     accuracy = accuracy_score(y_test.values.ravel(), y_pred)
#     precision = precision_score(y_test.values.ravel(), y_pred)
#     recall = recall_score(y_test.values.ravel(), y_pred)
#     f1 = f1_score(y_test.values.ravel(), y_pred)

#     mlflow.log_metric('accuracy', accuracy)
#     mlflow.log_metric('precision', precision)
#     mlflow.log_metric('recall', recall)
#     mlflow.log_metric('f1', f1)

#     from mlflow.sklearn import save_model
#     save_model(model, "model")
#     mlflow.log_artifact("model/model.pkl", artifact_path="model")
#     mlflow.log_artifact("model/MLmodel", artifact_path="model")

    
#     import os
#     notebook_path = "exp_bm.ipynb"
#     os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
#     mlflow.log_artifact(notebook_path)

#     print(f"Accuracy: {accuracy}")
#     print(f"Precision: {precision}")
#     print(f"Recall: {recall}")
#     print(f"F1 Score: {f1}")





import os
import datetime
import mlflow
from mlflow.sklearn import save_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

with mlflow.start_run():
    mlflow.log_param('vectorizer', 'Bag of Words')
    mlflow.log_param('num_features', 1000)
    mlflow.log_param('test_size', 0.2)

    model = LogisticRegression()
    model.fit(X_train, y_train.values.ravel()) 

    mlflow.log_param('model', 'Logistic Regression')

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test.values.ravel(), y_pred)
    precision = precision_score(y_test.values.ravel(), y_pred)
    recall = recall_score(y_test.values.ravel(), y_pred)
    f1 = f1_score(y_test.values.ravel(), y_pred)

    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('precision', precision)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('f1', f1)

    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    model_path = f"model_{timestamp}"

    save_model(model, model_path)

    for root, dirs, files in os.walk(model_path):
        for file in files:
            full_path = os.path.join(root, file)
            relative_path = os.path.relpath(full_path, model_path)
            mlflow.log_artifact(full_path, artifact_path=os.path.join("model", os.path.dirname(relative_path)))

    notebook_path = "exp_bm.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")



Accuracy: 0.7787951807228916
Precision: 0.765774378585086
Recall: 0.7891625615763547
F1 Score: 0.777292576419214
🏃 View run resilient-eel-459 at: https://dagshub.com/KARAN-sys141/mini-mlops.mlflow/#/experiments/0/runs/ce78fe283ba14cb0b0bbb26db0d9739a
🧪 View experiment at: https://dagshub.com/KARAN-sys141/mini-mlops.mlflow/#/experiments/0
